In [48]:
!pip install kagglehub pandas matplotlib python-pptx google-genai python-dotenv

In [49]:
import os
from dataclasses import dataclass
from typing import Dict, List

import kagglehub
import pandas as pd
import matplotlib.pyplot as plt

from pptx import Presentation
from pptx.util import Inches, Pt

from google import genai
from dotenv import load_dotenv

load_dotenv()

# Plot + pandas settings
plt.rcParams["figure.figsize"] = (8, 4)
pd.set_option("display.max_columns", 50)

# Output paths
OUTPUT_DIR = "output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

PPTX_PATH = os.path.join(OUTPUT_DIR, "Marketing_Insight_Engine_Deck.pptx")
CHART_CHANNEL_PATH = os.path.join(OUTPUT_DIR, "uplift_by_channel.png")
CHART_OBJECTIVE_PATH = os.path.join(OUTPUT_DIR, "uplift_by_objective.png")

GEMINI_MODEL = "gemini-2.5-flash"


In [50]:
# Download latest version of the dataset from Kaggle
path = kagglehub.dataset_download("geethasagarbonthu/marketing-and-e-commerce-analytics-dataset")
print("Path to dataset files:", path)

# See what files we have
files = []
for root, dirs, fs in os.walk(path):
    for f in fs:
        files.append(os.path.join(root, f))

files

Path to dataset files: C:\Users\adity\.cache\kagglehub\datasets\geethasagarbonthu\marketing-and-e-commerce-analytics-dataset\versions\1


['C:\\Users\\adity\\.cache\\kagglehub\\datasets\\geethasagarbonthu\\marketing-and-e-commerce-analytics-dataset\\versions\\1\\campaigns.csv',
 'C:\\Users\\adity\\.cache\\kagglehub\\datasets\\geethasagarbonthu\\marketing-and-e-commerce-analytics-dataset\\versions\\1\\customers.csv',
 'C:\\Users\\adity\\.cache\\kagglehub\\datasets\\geethasagarbonthu\\marketing-and-e-commerce-analytics-dataset\\versions\\1\\events.csv',
 'C:\\Users\\adity\\.cache\\kagglehub\\datasets\\geethasagarbonthu\\marketing-and-e-commerce-analytics-dataset\\versions\\1\\products.csv',
 'C:\\Users\\adity\\.cache\\kagglehub\\datasets\\geethasagarbonthu\\marketing-and-e-commerce-analytics-dataset\\versions\\1\\transactions.csv']

In [51]:
def pick_campaign_csv(file_list: List[str]) -> str:
    """
    Pick the CSV that looks like the marketing campaign file.
    Priority:
      1. filename containing 'campaign'
      2. otherwise first CSV
    """
    csv_files = [f for f in file_list if f.lower().endswith(".csv")]
    if not csv_files:
        raise FileNotFoundError("No CSV files found in the dataset.")

    for f in csv_files:
        name = os.path.basename(f).lower()
        if "campaign" in name:
            return f

    return csv_files[0]


campaign_csv_path = pick_campaign_csv(files)
print("Using campaign CSV:", campaign_csv_path)

df_raw = pd.read_csv(campaign_csv_path)
df_raw.head()

Using campaign CSV: C:\Users\adity\.cache\kagglehub\datasets\geethasagarbonthu\marketing-and-e-commerce-analytics-dataset\versions\1\campaigns.csv


,campaign_id,channel,objective,start_date,end_date,target_segment,expected_uplift
0,1,Paid Search,Cross-sell,2021-10-25,2021-11-26,Deal Seekers,0.022
1,2,Email,Retention,2021-10-24,2021-12-24,Deal Seekers,0.116
2,3,Email,Reactivation,2023-10-08,2023-11-30,Churn Risk,0.100
3,4,Display,Reactivation,2022-07-25,2022-10-07,Deal Seekers,0.111
4,5,Social,Acquisition,2022-07-09,2022-09-29,New Customers,0.144


In [52]:
@dataclass
class SummaryStats:
    overall: pd.Series
    by_channel: pd.DataFrame
    by_objective: pd.DataFrame
    by_segment: pd.DataFrame
    by_start_month: pd.DataFrame


def load_and_clean(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean and type-cast the campaign data.

    Expected columns (based on dataset inspection):
      - campaign_id
      - channel
      - objective
      - start_date
      - end_date
      - target_segment
      - expected_uplift
    """
    df = df.copy()
    df.columns = [c.strip().lower() for c in df.columns]

    expected = [
        "campaign_id", "channel", "objective",
        "start_date", "end_date",
        "target_segment", "expected_uplift"
    ]
    missing = [c for c in expected if c not in df.columns]
    if missing:
        raise ValueError(
            f"Missing expected columns: {missing}\n"
            f"Columns found: {list(df.columns)}"
        )

    # Parse dates
    df["start_date"] = pd.to_datetime(df["start_date"], errors="coerce")
    df["end_date"] = pd.to_datetime(df["end_date"], errors="coerce")

    # Numeric uplift
    df["expected_uplift"] = pd.to_numeric(df["expected_uplift"], errors="coerce")

    # Drop rows with missing key fields
    df = df.dropna(subset=[
        "campaign_id", "channel", "objective",
        "start_date", "end_date", "target_segment",
        "expected_uplift"
    ])

    return df


df = load_and_clean(df_raw)
df.head()

,campaign_id,channel,objective,start_date,end_date,target_segment,expected_uplift
0,1,Paid Search,Cross-sell,2021-10-25,2021-11-26,Deal Seekers,0.022
1,2,Email,Retention,2021-10-24,2021-12-24,Deal Seekers,0.116
2,3,Email,Reactivation,2023-10-08,2023-11-30,Churn Risk,0.100
3,4,Display,Reactivation,2022-07-25,2022-10-07,Deal Seekers,0.111
4,5,Social,Acquisition,2022-07-09,2022-09-29,New Customers,0.144


In [53]:
def engineer_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add helpful metrics like campaign duration and start month.
    """
    df = df.copy()
    df["duration_days"] = (df["end_date"] - df["start_date"]).dt.days

    # Remove negative durations if any
    df = df[df["duration_days"] >= 0]

    # Start month (YYYY-MM)
    df["start_month"] = df["start_date"].dt.to_period("M").astype(str)

    return df


def build_summary_tables(df: pd.DataFrame) -> SummaryStats:
    """
    Aggregate by channel, objective, segment, and month.
    """
    overall = pd.Series({
        "Total_Campaigns": len(df),
        "Avg_Expected_Uplift": df["expected_uplift"].mean(),
        "Median_Expected_Uplift": df["expected_uplift"].median(),
        "Max_Expected_Uplift": df["expected_uplift"].max(),
        "Min_Expected_Uplift": df["expected_uplift"].min(),
        "Avg_Duration_Days": df["duration_days"].mean(),
        "Total_Days_Run": df["duration_days"].sum()
    })

    by_channel = (
        df.groupby("channel")
        .agg(
            Campaigns=("campaign_id", "count"),
            Avg_Expected_Uplift=("expected_uplift", "mean"),
            Median_Expected_Uplift=("expected_uplift", "median"),
            Max_Expected_Uplift=("expected_uplift", "max"),
            Avg_Duration_Days=("duration_days", "mean"),
        )
        .sort_values("Avg_Expected_Uplift", ascending=False)
    )

    by_objective = (
        df.groupby("objective")
        .agg(
            Campaigns=("campaign_id", "count"),
            Avg_Expected_Uplift=("expected_uplift", "mean"),
            Median_Expected_Uplift=("expected_uplift", "median"),
            Max_Expected_Uplift=("expected_uplift", "max"),
            Avg_Duration_Days=("duration_days", "mean"),
        )
        .sort_values("Avg_Expected_Uplift", ascending=False)
    )

    by_segment = (
        df.groupby("target_segment")
        .agg(
            Campaigns=("campaign_id", "count"),
            Avg_Expected_Uplift=("expected_uplift", "mean"),
            Median_Expected_Uplift=("expected_uplift", "median"),
            Max_Expected_Uplift=("expected_uplift", "max"),
        )
        .sort_values("Avg_Expected_Uplift", ascending=False)
    )

    by_start_month = (
        df.groupby("start_month")
        .agg(
            Campaigns=("campaign_id", "count"),
            Avg_Expected_Uplift=("expected_uplift", "mean"),
        )
        .sort_index()
    )

    return SummaryStats(
        overall=overall,
        by_channel=by_channel,
        by_objective=by_objective,
        by_segment=by_segment,
        by_start_month=by_start_month,
    )


df = engineer_features(df)
summary = build_summary_tables(df)

summary.overall

Total_Campaigns             50.00000
Avg_Expected_Uplift          0.08666
Median_Expected_Uplift       0.09250
Max_Expected_Uplift          0.14400
Min_Expected_Uplift          0.02200
Avg_Duration_Days           50.98000
Total_Days_Run            2549.00000
dtype: float64

In [54]:
def create_channel_chart(by_channel: pd.DataFrame, chart_path: str) -> None:
    """
    Bar chart: average expected uplift by channel.
    """
    plt.figure(figsize=(8, 4))
    by_channel["Avg_Expected_Uplift"].plot(kind="bar")
    plt.title("Average Expected Uplift by Channel")
    plt.ylabel("Avg Expected Uplift")
    plt.tight_layout()
    plt.savefig(chart_path)
    plt.close()


def create_objective_chart(by_objective: pd.DataFrame, chart_path: str) -> None:
    """
    Bar chart: average expected uplift by campaign objective.
    """
    plt.figure(figsize=(8, 4))
    by_objective["Avg_Expected_Uplift"].plot(kind="bar")
    plt.title("Average Expected Uplift by Objective")
    plt.ylabel("Avg Expected Uplift")
    plt.tight_layout()
    plt.savefig(chart_path)
    plt.close()


create_channel_chart(summary.by_channel, CHART_CHANNEL_PATH)
create_objective_chart(summary.by_objective, CHART_OBJECTIVE_PATH)

CHART_CHANNEL_PATH, CHART_OBJECTIVE_PATH

('output\\uplift_by_channel.png', 'output\\uplift_by_objective.png')

In [55]:
def stats_to_text(summary: SummaryStats) -> str:
    """
    Turn numeric summaries into a text block for the LLM.
    """
    o = summary.overall

    top_channels = summary.by_channel.head(5).reset_index()
    top_objectives = summary.by_objective.head(5).reset_index()
    top_segments = summary.by_segment.head(5).reset_index()

    lines = []

    lines.append("OVERALL CAMPAIGN PERFORMANCE")
    lines.append(
        f"Total campaigns: {o['Total_Campaigns']:.0f}, "
        f"Average expected uplift: {o['Avg_Expected_Uplift']:.2f}, "
        f"Median uplift: {o['Median_Expected_Uplift']:.2f}."
    )
    lines.append(
        f"Max uplift: {o['Max_Expected_Uplift']:.2f}, "
        f"Min uplift: {o['Min_Expected_Uplift']:.2f}, "
        f"Average campaign duration: {o['Avg_Duration_Days']:.1f} days."
    )

    lines.append("\nTOP CHANNELS BY EXPECTED UPLIFT")
    for _, row in top_channels.iterrows():
        lines.append(
            f"- {row['channel']}: "
            f"Avg uplift {row['Avg_Expected_Uplift']:.2f}, "
            f"Median uplift {row['Median_Expected_Uplift']:.2f}, "
            f"Max uplift {row['Max_Expected_Uplift']:.2f}, "
            f"Campaigns {row['Campaigns']}."
        )

    lines.append("\nTOP OBJECTIVES BY EXPECTED UPLIFT")
    for _, row in top_objectives.iterrows():
        lines.append(
            f"- {row['objective']}: "
            f"Avg uplift {row['Avg_Expected_Uplift']:.2f}, "
            f"Median uplift {row['Median_Expected_Uplift']:.2f}, "
            f"Max uplift {row['Max_Expected_Uplift']:.2f}, "
            f"Campaigns {row['Campaigns']}."
        )

    lines.append("\nTOP TARGET SEGMENTS BY EXPECTED UPLIFT")
    for _, row in top_segments.iterrows():
        lines.append(
            f"- {row['target_segment']}: "
            f"Avg uplift {row['Avg_Expected_Uplift']:.2f}, "
            f"Median uplift {row['Median_Expected_Uplift']:.2f}, "
            f"Campaigns {row['Campaigns']}."
        )

    return "\n".join(lines)


# Preview what we will send to Gemini
stats_preview = stats_to_text(summary)
print(stats_preview[:800], "...\n")

OVERALL CAMPAIGN PERFORMANCE
Total campaigns: 50, Average expected uplift: 0.09, Median uplift: 0.09.
Max uplift: 0.14, Min uplift: 0.02, Average campaign duration: 51.0 days.

TOP CHANNELS BY EXPECTED UPLIFT
- Affiliate: Avg uplift 0.10, Median uplift 0.10, Max uplift 0.14, Campaigns 11.
- Paid Search: Avg uplift 0.09, Median uplift 0.10, Max uplift 0.14, Campaigns 11.
- Display: Avg uplift 0.09, Median uplift 0.10, Max uplift 0.14, Campaigns 9.
- Email: Avg uplift 0.07, Median uplift 0.09, Max uplift 0.14, Campaigns 11.
- Social: Avg uplift 0.07, Median uplift 0.06, Max uplift 0.14, Campaigns 8.

TOP OBJECTIVES BY EXPECTED UPLIFT
- Retention: Avg uplift 0.09, Median uplift 0.11, Max uplift 0.14, Campaigns 13.
- Cross-sell: Avg uplift 0.09, Median uplift 0.10, Max uplift 0.14, Campaigns 1 ...



In [56]:
def create_gemini_client() -> genai.Client:
    """
    Create a Gemini client. The SDK will read GEMINI_API_KEY from env by default,
    or you can pass api_key=... explicitly.
    """
    api_key = os.getenv("GEMINI_API_KEY")
    if not api_key:
        raise RuntimeError("Please set GEMINI_API_KEY in your environment or .env file.")
    client = genai.Client(api_key=api_key)
    return client


def generate_narrative_with_gemini(client: genai.Client, summary: SummaryStats) -> Dict[str, str]:
    """
    Use Gemini to create an executive summary and recommendations
    based on expected uplift across channels/objectives/segments.
    """
    stats_text = stats_to_text(summary)

    prompt = f"""
You are a senior marketing analytics consultant.

Here are aggregated metrics from a campaign performance dataset
(where 'expected_uplift' is the main success metric):

{stats_text}

Write three sections:

1) Executive Summary (3–5 short bullet points)
2) Channel, Objective & Segment Insights (3–5 short bullet points)
3) Recommendations to improve expected uplift (3–5 short bullet points)

Guidelines:
- Use short, clear bullet points.
- Avoid technical jargon.
- Focus on which channels/objectives/segments are strong or weak,
  and what actions to take.
"""

    response = client.models.generate_content(
        model=GEMINI_MODEL,
        contents=prompt,
    )

    content = response.text
    return {
        "full_narrative": content,
        "stats_text": stats_text,
    }

In [ ]:
def add_title_slide(prs: Presentation) -> None:
    slide_layout = prs.slide_layouts[0]  # title slide
    slide = prs.slides.add_slide(slide_layout)
    slide.shapes.title.text = "Automated Campaign Insight Engine"
    subtitle = slide.placeholders[1]
    subtitle.text = "Auto-generated deck from Marketing Campaign Uplift Data"


def add_overall_slide(prs: Presentation, summary: SummaryStats) -> None:
    slide_layout = prs.slide_layouts[1]  # title + content
    slide = prs.slides.add_slide(slide_layout)
    slide.shapes.title.text = "Overall Campaign Performance"

    body = slide.placeholders[1].text_frame
    body.clear()

    o = summary.overall
    lines = [
        f"Total campaigns: {o['Total_Campaigns']:.0f}",
        f"Average expected uplift: {o['Avg_Expected_Uplift']:.2f}",
        f"Median uplift: {o['Median_Expected_Uplift']:.2f}",
        f"Max uplift: {o['Max_Expected_Uplift']:.2f}",
        f"Min uplift: {o['Min_Expected_Uplift']:.2f}",
        f"Average duration: {o['Avg_Duration_Days']:.1f} days",
    ]

    for i, line in enumerate(lines):
        if i == 0:
            p = body.paragraphs[0]
        else:
            p = body.add_paragraph()
            p.level = 1
        p.text = line
        p.font.size = Pt(20)


def add_channel_table_slide(prs: Presentation, summary: SummaryStats) -> None:
    slide_layout = prs.slide_layouts[5]  # title only
    slide = prs.slides.add_slide(slide_layout)
    slide.shapes.title.text = "Top Channels by Expected Uplift"

    df_ch = summary.by_channel.head(5)

    rows = len(df_ch) + 1
    cols = 5
    left = Inches(0.5)
    top = Inches(1.5)
    width = Inches(9)
    height = Inches(1.0)

    table = slide.shapes.add_table(rows, cols, left, top, width, height).table

    headers = ["Channel", "Campaigns", "Avg Uplift", "Median Uplift", "Max Uplift"]
    for i, h in enumerate(headers):
        cell = table.cell(0, i)
        cell.text = h
        cell.text_frame.paragraphs[0].font.bold = True
        cell.text_frame.paragraphs[0].font.size = Pt(16)

    for r, (channel, row) in enumerate(df_ch.iterrows(), start=1):
        table.cell(r, 0).text = str(channel)
        table.cell(r, 1).text = f"{int(row['Campaigns'])}"
        table.cell(r, 2).text = f"{row['Avg_Expected_Uplift']:.2f}"
        table.cell(r, 3).text = f"{row['Median_Expected_Uplift']:.2f}"
        table.cell(r, 4).text = f"{row['Max_Expected_Uplift']:.2f}"

        for c in range(cols):
            for p in table.cell(r, c).text_frame.paragraphs:
                p.font.size = Pt(14)


def add_chart_slide(prs: Presentation, chart_path: str, title: str) -> None:
    if not os.path.exists(chart_path):
        return
    slide_layout = prs.slide_layouts[5]
    slide = prs.slides.add_slide(slide_layout)
    slide.shapes.title.text = title
    left = Inches(0.8)
    top = Inches(1.5)
    slide.shapes.add_picture(chart_path, left, top, height=Inches(4.5))


def _split_narrative_sections(narrative_text: str):
    
    text = narrative_text.strip()

    if "###" in text:
        parts = [p.strip() for p in text.split("###") if p.strip()]
        sections = {}
        for p in parts:
            lines = p.splitlines()
            title = lines[0].strip()
            body = "\n".join(lines[1:]).strip()
            sections[title] = body

        return [
            ("Executive Summary", sections.get("1) Executive Summary", text)),
            ("Channel, Objective & Segment Insights", sections.get("2) Channel, Objective & Segment Insights", text)),
            ("Recommendations to Improve Uplift", sections.get("3) Recommendations to improve expected uplift", text)),
        ]

    # Fallback: no markdown headings → just put everything as one section
    return [("AI-Generated Executive Insights", text)]


def add_llm_slides(prs: Presentation, narrative_text: str) -> None:
    """
    Create up to 3 slides:
      - Executive Summary
      - Insights
      - Recommendations
    Each slide gets short bullets so text doesn't overflow.
    """
    sections = _split_narrative_sections(narrative_text)

    for title, body in sections:
        slide_layout = prs.slide_layouts[1]  # title + content
        slide = prs.slides.add_slide(slide_layout)
        slide.shapes.title.text = title

        tf = slide.placeholders[1].text_frame
        tf.clear()

        lines = [l.strip("-* ").strip() for l in body.splitlines() if l.strip()]

        lines = lines[:8]

        for i, line in enumerate(lines):
            if not line:
                continue
            if i == 0:
                p = tf.paragraphs[0]
            else:
                p = tf.add_paragraph()
            p.text = line
            p.level = 0
            p.font.size = Pt(20)
            p.font.bold = False


def generate_pptx(summary: SummaryStats, narrative: Dict[str, str]) -> None:
    prs = Presentation()
    add_title_slide(prs)
    add_overall_slide(prs, summary)
    add_channel_table_slide(prs, summary)
    add_chart_slide(prs, CHART_CHANNEL_PATH, "Average Expected Uplift by Channel")
    add_chart_slide(prs, CHART_OBJECTIVE_PATH, "Average Expected Uplift by Objective")
    add_llm_slides(prs, narrative["full_narrative"])

    prs.save(PPTX_PATH)
    print(f"PowerPoint deck saved to: {PPTX_PATH}")


In [58]:
use_gemini = True

if use_gemini:
    try:
        client = create_gemini_client()
        narrative = generate_narrative_with_gemini(client, summary)
    except Exception as e:
        print("Gemini call failed, using placeholder narrative instead.")
        print("Error:", e)
        placeholder_text = (
            "Executive Summary\n"
            "- This slide deck was generated automatically from the campaign dataset.\n"
            "- Channels and objectives with higher expected uplift should be prioritized.\n"
            "\nChannel, Objective & Segment Insights\n"
            "- See earlier slides for top-performing channels and segments.\n"
            "\nRecommendations\n"
            "- Focus budget on channels and segments with the highest uplift.\n"
            "- Experiment with underperforming areas using small test budgets.\n"
        )
        narrative = {
            "full_narrative": placeholder_text,
            "stats_text": stats_to_text(summary)
        }
else:
    # Skip Gemini completely and just use a static narrative
    placeholder_text = (
        "Executive Summary\n"
        "- This slide deck was generated automatically from the campaign dataset.\n"
        "- Channels and objectives with higher expected uplift should be prioritized.\n"
        "\nChannel, Objective & Segment Insights\n"
        "- See earlier slides for top-performing channels and segments.\n"
        "\nRecommendations\n"
        "- Focus budget on channels and segments with the highest uplift.\n"
        "- Experiment with underperforming areas using small test budgets.\n"
    )
    narrative = {
        "full_narrative": placeholder_text,
        "stats_text": stats_to_text(summary)
    }

generate_pptx(summary, narrative)

PowerPoint deck saved to: output\Marketing_Insight_Engine_Deck.pptx
